In [1]:
# 데이터 크롤링 모듈
!pip install selenium
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
from functools import reduce

     |████████████████████████████████| 911kB 7.8MB/s 


In [2]:
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:7 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [56.8 kB]
Get:9 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:10 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Ign:13 https://developer.download.nvi

In [3]:

chrome_options = webdriver.ChromeOptions()

chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정

chrome_options.add_argument('--no-sandbox')

chrome_options.add_argument('--disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver',options=chrome_options)

In [105]:
def create_table(column):
    df = []
    df = pd.DataFrame(columns=column)
    html = driver.page_source
    bsObject = BeautifulSoup(html, 'html.parser')
    temp = bsObject.find_all("table")[1]
    templen = len(temp.find_all("tr"))
    
    for i in range(2, templen):
        tempTr = temp.find_all("tr")[i]
        if(tempTr.find("th") is not None):
            continue
        row = {}
        column_idx = 0
        for j in range(len(column)):
            tempTd = tempTr.find_all("td")[j].text
            if(tempTd == "" and j > 0):
                continue
            row[column[column_idx]] = tempTd
            column_idx += 1
        df = df.append(row,ignore_index=True)

    html = driver.find_element_by_xpath('//*[@id="mytable"]/tbody').get_attribute("innerHTML") #기록 table을 str형태로 저장
    soup = BeautifulSoup(html, 'html.parser') 
    
    href = [i.find("a") for i in soup.findAll('tr') if 'birth' in i.find('a').attrs['href']]
    p = re.compile("\d{4}\-\d{2}\-\d{2}")
    birth = [p.findall(i.attrs['href'])[0] for i in href]
    birth = pd.DataFrame(birth, columns = ["birth"])
    tag = pd.DataFrame(columns = ['tag'])
    tag["tag"] = df["이름"] + birth["birth"]
    df = df.replace('  ',0)
    return pd.concat([df, tag], axis=1)

In [60]:
#Bat base
driver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=auto&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR_ALL_ADJ&o2=TPA&de=1&lr=0&tr=&cv=&ml=1&sn=234")

column = ["순","이름","팀","WAR","G","타석","타수","득점","안타","2타","3타","홈런","루타","타점","도루","도실","볼넷","사구","삼진","병살","희타","희비",
          "WAR+","타율","출루","장타","OPS","wOBA","wRC+","WAR2","WPA"]

In [83]:
bat_base = create_table(column)
bat_base = bat_base[['이름', '팀','WAR','G','OPS','wOBA', 'wRC+', 'WPA','tag']]

In [84]:
driver.get('http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=WRCPLUS&de=1&o2=WAR_ALL&lr=0&tr=&cv=&ml=1&sn=234')

column = ["순","이름","팀","wRC+","타석","HR%","BB%","K%","BB/K","IsoP","IsoD","BABIP","Spd","PSN","wOBA",
          "wRC","wRC/27","wRAA","wOBA2","wRC2","wRC/272","wRAA","wRC+2"]
bat_plus = create_table(column)

In [85]:
bat_plus = bat_plus[['BB/K','IsoP','IsoD','BABIP','Spd','tag']]

In [86]:
driver.get('http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=4&o1=WPA&de=1&o2=WAR_ALL&lr=0&tr=&cv=&ml=1&sn=234')

column = ["순","이름","팀","WPA","PA","PA%","PA타율","타점","순수","주자","타점%","RE타격","RE주루","RE24","REW",
          "WPA타격","WPA주루","WPA+","WPA-","WPA2","pLI","WPA/LI","Clutch","ph","phLI"]
bat_clutch = create_table(column)

In [87]:
bat_clutch = bat_clutch[['pLI','Clutch','tag']]

In [88]:
driver.get('http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=0&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=5&o1=PPA&de=1&o2=WAR_ALL&lr=0&tr=&cv=&ml=1&sn=234')

column = ["순","이름","팀","P/PA","타석","투구","P/PA2","루킹","헛스윙","파울","타격","S%","B%","배트 전체","초구",
          "컨택%","2S후C&","2S후선구%","스윙삼진","루킹삼진","LSO%","N","스윙","W%"]
bat_atbat = create_table(column)

In [89]:
bat_atbat = bat_atbat[['P/PA','tag']]

In [99]:
dfs = [bat_base, bat_plus, bat_clutch, bat_atbat]
batter = reduce(lambda left, right: pd.merge(left, right, on='tag'), dfs)

In [100]:
batter['팀'] = np.where(batter['팀'].str[2] == 'N', 'NC', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == '두', '두산', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == '키', '키움', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == 'L', 'LG', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == 'K', 'K', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == '롯', 'NC', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == '삼', '삼성', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == 'S', 'SSG', batter['팀'])
batter['팀'] = np.where(batter['팀'].str[2] == '한', '한화', batter['팀'])

In [102]:
batter = batter.drop(['tag'], axis=1)

In [103]:
# Result
batter

,이름,팀,WAR,G,OPS,wOBA,wRC+,WPA,BB/K,IsoP,IsoD,BABIP,Spd,pLI,Clutch,P/PA
0,알테어,NC,1.99,35,1.101,.481,178.8,1.10,0.31,.355,.068,.385,4.1,1.10,1.51,3.72
1,이정후,키움,1.92,37,.939,.442,159.0,0.34,1.85,.134,.101,.385,5.9,0.34,1.10,4.02
2,피렐라,삼성,1.81,36,1.057,.468,171.5,1.74,0.65,.290,.050,.348,4.9,1.74,1.64,3.72
3,강백호,K,1.81,35,1.049,.475,178.8,1.63,1.12,.183,.064,.424,3.7,1.63,1.55,4.07
4,양의지,NC,1.73,34,1.070,.477,178.2,1.59,1.54,.282,.115,.330,4.4,1.59,1.19,3.62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
229,김동엽,삼성,-0.47,11,.330,.176,-15.1,-0.34,0.30,.000,.080,.160,3.3,-0.34,-0.45,4.10
230,박정현,한화,-0.49,27,.494,.237,25.1,-0.43,0.25,.032,.053,.288,2.7,-0.43,-0.57,3.64
231,도태훈,NC,-0.50,16,.192,.108,-60.4,-0.51,0.17,.000,.058,.083,2.7,-0.51,-0.55,3.94
232,이성열,한화,-0.50,21,.466,.246,31.2,0.01,0.53,.019,.115,.237,3.1,0.01,-0.33,3.92


In [122]:
batter.to_csv('batter.csv',encoding='utf-8-sig', index = False)

In [108]:
#Pit base
driver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=1&o1=WAR&o2=OutCount&de=1&lr=0&tr=&cv=&ml=1&sn=216")

column = ["순","이름","팀","WAR","출장","완투","완봉","선발","승","패","세","홀드","이닝","실점","자책","타자","안타","2타","3타","홈런","볼넷","고4",
          "사구","삼진","보크","폭투","ERA","FIP","WHIP","ERA+","FIP+","WAR2","WPA"]
pit_base = create_table(column)

In [109]:
pit_base = pit_base[['이름', '팀','WAR','ERA','WHIP', 'WPA','tag']]

In [110]:
driver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=2&o1=FIP&de=0&o2=WAR&lr=0&tr=&cv=&ml=1&sn=216")

column = ["순","이름","팀","FIP","출장","이닝","ERA","FIP2","K/9","BB/9","K/BB","HR/9","K%","BB%","K-BB%","PFR","BABIP","LOB%","타율","출루율","장타율","OPS",
          "WHIP","WHIP+","투구","IP/G","P/G","P/IP","P/PA","CYP"]
pit_plus = create_table(column)

In [111]:
pit_plus = pit_plus[['K-BB%','BABIP','LOB%','IP/G', 'P/PA','tag']]

In [112]:
driver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=4&o1=WAR_ALL&de=1&o2=WAR_ALL&lr=0&tr=&cv=&ml=1&sn=216&")

column = ["순","이름","팀","WAR","출장","이닝","RAA선발","RAA구원","RAA","대체선발","대체구원","대체종합","RAR선발","RAR구원","RAR","WAA선발","WAA구원","WAA","WAR선발","WAR구원","WAR2","fWAR",
          "RA9-WAR"]
pit_worth = create_table(column)

In [113]:
pit_worth = pit_worth[['RAA','RAR','WAA','tag']]

In [114]:
driver.get("http://www.statiz.co.kr/stat.php?opt=0&sopt=0&re=1&ys=2021&ye=2021&se=0&te=&tm=&ty=0&qu=all&po=0&as=&ae=&hi=&un=&pl=&da=13&o1=wFB&de=1&o2=WAR&lr=0&tr=&cv=&ml=1&sn=216")

column = ["순","이름","팀","가치","출장","이닝","직구","슬라이더","커브","체인지업","스플리터","싱커","너클","기타","직","슬","커","체","스","싱","너","기"]
pit_value = create_table(column)

In [115]:
pit_value = pit_value[['직구','슬라이더', '커브', '체인지업','스플리터','싱커','너클','tag']]

In [116]:
dfs = [pit_base, pit_plus, pit_worth, pit_value]
pitcher = reduce(lambda left, right: pd.merge(left, right, on='tag'), dfs)

In [117]:
pitcher['팀'] = np.where(pitcher['팀'].str[2] == 'N', 'NC', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == '두', '두산', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == '키', '키움', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == 'L', 'LG', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == 'K', 'K', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == '롯', 'NC', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == '삼', '삼성', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == 'S', 'SSG', pitcher['팀'])
pitcher['팀'] = np.where(pitcher['팀'].str[2] == '한', '한화', pitcher['팀'])

In [119]:
pitcher = pitcher.drop(['tag'], axis=1)

In [120]:
# Result
pitcher

,이름,팀,WAR,ERA,WHIP,WPA,K-BB%,BABIP,LOB%,IP/G,P/PA,RAA,RAR,WAA,직구,슬라이더,커브,체인지업,스플리터,싱커,너클
0,원태인,삼성,3.16,1.00,1.00,2.00,19.7,.282,89.1,6.4,3.80,21.85,27.93,2.56,0.8,5.7,2.4,11.6,0.0,0.0,0.0
1,뷰캐넌,삼성,2.08,2.17,1.15,0.81,15.8,.287,77.5,6.2,3.95,14.26,21.85,1.42,2.5,4.3,0.6,4.0,0.0,1.1,0.0
2,로켓,두산,1.76,2.34,1.37,0.70,6.5,.309,84.2,6.1,3.88,12.12,18.62,1.20,0.6,0.1,0.2,5.1,0.0,6.4,0.0
3,박종훈,SSG,1.62,2.68,1.10,0.80,10.6,.256,73.1,6.2,3.98,10.82,17.63,1.04,1.2,0.0,4.7,0.9,0.0,2.2,0.0
4,수아레즈,LG,1.54,1.96,1.11,1.18,20.4,.292,81.1,5.9,3.96,10.37,16.85,0.99,3.9,4.6,-0.6,0.1,0.0,3.3,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,쿠에바스,K,-0.64,7.04,1.91,-1.01,6.0,.349,52.8,4.6,3.85,-10.79,-6.16,-0.94,-0.2,-2.9,-1.1,-6.9,0.0,0.1,0.0
212,장재영,키움,-0.64,16.50,2.33,-0.50,-11.8,.278,31.3,0.9,4.82,-8.03,-7.28,-0.72,-3.0,-0.5,-1.1,-0.1,0.0,0.0,0.0
213,임찬규,LG,-0.67,21.21,3.86,-0.75,-19.4,.526,33.3,2.3,4.26,-10.16,-9.07,-0.73,-6.1,0.0,0.8,-1.6,0.0,0.0,0.0
214,이준영,K,-0.72,13.50,2.18,-0.33,-7.9,.292,37.9,0.4,4.03,-6.55,-6.66,-0.81,-3.1,1.2,-2.7,0.0,0.0,0.0,0.0


In [123]:
pitcher.to_csv('pitcher.csv',encoding='utf-8-sig', index = False)